In [1]:
import datetime
import os
import shutil

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.metrics import MeanAbsolutePercentageError, MeanAbsoluteError, RootMeanSquaredError

import definitions
from training import train, data
from training.loguniform import LogUniform
from training.stepuniform import StepUniform
from training.steploguniform import StepLogUniform
from scipy.stats.distributions import randint
import numpy as np
import pandas as pd

import altair as alt

alt.data_transformers.enable('data_server')
#alt.data_transformers.disable_max_rows()

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

C:\Users\alexj\.virtualenvs\mass_regression-87a6zBVK\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Missing Mass Regression
## H -> WW -> lnulnu

In [2]:
dataset = 'H125'
target = 'H'

In [8]:
jigsaw_train, jigsaw_val, jigsaw_test = data.get_jigsaw(dataset=dataset, target=target)
x_train, y_train, x_val, y_val, x_test, y_test = data.get_datasets(dataset=dataset, target=target, scale=True)
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000)
print("x_train:")
print(x_train)
print(f"num training samples: {x_train.shape[0]}")
print(f"num validation samples: {x_val.shape[0]}")
print(f"num testing samples: {x_test.shape[0]}")
print(y_train)

x_train:
           METx      METy  Lax_reco  Lay_reco  Laz_reco  Lam_reco  Lbx_reco  Lby_reco  Lbz_reco  Lbm_reco
25499 -0.250382 -0.296569  0.747626  0.136942 -2.787683  0.000505 -0.503431  0.153440 -0.404184  0.000505
40218  0.917223  0.350753 -1.056465 -0.561564 -4.083565  0.000505  0.133056  0.204624 -0.214510  0.000505
12536  0.497515 -0.969540  0.217005  0.318689 -0.864559  0.000505 -0.720707  0.644664 -1.704122  0.000505
4878  -0.498057 -0.148172  0.304802 -0.304384  1.953907  0.000505  0.187068  0.446369  0.193224  0.000505
16817  0.716234 -0.713684 -0.527178 -0.043852  0.856596  0.000505 -0.195243  0.751350  1.197897  0.000505
...         ...       ...       ...       ...       ...       ...       ...       ...       ...       ...
92127 -0.846182 -0.359765  0.485321 -0.027004  0.741265  0.000505  0.354674  0.380582 -0.084807  0.000505
46693 -0.883018 -0.147421  0.241812 -0.209974  0.368327  0.000505  0.635020  0.351208  0.217983  0.000505
45699  0.070536 -0.100055  0.398308 -

## Dataset
Simple H -> WW -> lnulnu data samples are used. Below, the generated H mass (Hm_gen), jigsaw reconstructed H mass (Hm_reco) and the difference of the two are shown.

In [7]:
alt.Chart(y_test).mark_bar().encode(alt.X(f"{definitions.TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[120, 135], step=5)), y="count()")

alt.Chart(...)

In [5]:
alt.Chart(jigsaw_test).mark_bar().encode(alt.X(f"{definitions.JIGSAW_TARGETS[dataset][target][0]}:Q", bin=alt.Bin(extent=[0, 200], step=5)), y="count()")

alt.Chart(...)

In [6]:
jigsaw_difference = pd.DataFrame({'Wm_gen - Wm_reco': y_test.values[:, 0] - jigsaw_test[definitions.JIGSAW_TARGETS[dataset][target][0]].values})
alt.Chart(jigsaw_difference).mark_bar().encode(alt.X("Wm_gen - Wm_reco:Q", bin=alt.Bin(extent=[0, 100], step=5)), y="count()")

alt.Chart(...)

## Training
A simple fully-connected neural network is trained and tuned using random search.

In [7]:
def build_v2_model(hparams, input_shape):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    for _ in range(hparams['num_layers']):
        model.add(layers.Dense(units=hparams['num_units'],
                               activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        model.add(layers.Dropout(rate=hparams['dropout']))
    model.add(layers.Dense(1, dtype='float32'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hparams['learning_rate']),
        loss='mean_squared_error',
        metrics=[MeanAbsolutePercentageError(), MeanAbsoluteError(), RootMeanSquaredError()])
    return model

In [8]:
def random_search(build_fn, x, y, x_val, y_val, n, hp_rv, log_dir):
    best_model = None
    best_loss = float('Inf')
    for i in range(n):
        run_name = f'run{i}'
        run_dir = log_dir / run_name
        with tf.summary.create_file_writer(str(run_dir)).as_default():
            hparams = {k: v.rvs() for k, v in hp_rv.items()}
            hparams['run_num'] = i
            hp.hparams(hparams)
            model, loss, mape, mae, rmse = train_test_model(
                build_fn, x, y, x_val, y_val, hparams, run_dir)
            tf.summary.scalar('mean absolute percentage error', mape, step=1)
            tf.summary.scalar('mean absolute error', mae, step=1)
            tf.summary.scalar('root mean squared error', rmse, step=1)
            if loss < best_loss:
                best_model = model
                best_loss = loss
    best_model.save(str(log_dir / 'best_model.h5'))

In [9]:
hp_rv = {'num_layers': randint(1, 3),
             'num_units': StepUniform(start=10, num=20, step=10),
             'learning_rate': LogUniform(loc=-5, scale=4, base=10, discrete=False),
             'batch_size': StepLogUniform(start=5, num=4, step=1, base=2),
             'epochs': randint(10, 101),
             'dropout': StepUniform(start=0.0, num=2, step=0.5)}

## Results

In [10]:
log_dir = definitions.LOG_DIR / 'Wlnu' / 'v2'
model = tf.keras.models.load_model(str(log_dir / 'best_model.h5'))

In [11]:
y_pred = model.predict(x_val)[:,0]
y_test = y_test.values[:, 0]

In [12]:

chart_data = pd.DataFrame({'Wm_gen - Wm_reco': np.concatenate((y_test - y_pred, y_test - jigsaw_test[definitions.JIGSAW_TARGETS[dataset][target][0]].values)), 'Method': ['NN']*y_test.shape[0] + ['Jigsaw']*y_test.shape[0]})

alt.Chart(chart_data).mark_bar(opacity=0.7).encode(alt.X("Wm_gen - Wm_reco:Q", bin=alt.Bin(extent=[-40, 30], step=2)), y=alt.Y("count()", stack=None), color="Method")

alt.Chart(...)

In [13]:
print('Jigsaw:')
print('\tmae = ' + str(float(tf.keras.losses.MAE(y_test, jigsaw_test.values[:, 0]))))
print(f'\tmape = ' + str(float(tf.keras.losses.MAPE(y_test, jigsaw_test.values[:, 0]))))
print('\trmse = ' + str(float(tf.keras.losses.MSE(y_test, jigsaw_test.values[:, 0])**0.5)))
print('NN:')
print('\tmae = ' + str(float(tf.keras.losses.MAE(y_test, y_pred))))
print('\tmape = ' + str(float(tf.keras.losses.MAPE(y_test, y_pred))))
print('\trmse = ' + str(float(tf.keras.losses.MSE(y_test, y_pred)**0.5)))

Jigsaw:
	mae = 16.617048470402004
	mape = 20.40721814904974
	rmse = 25.327719402850715
NN:
	mae = 4.447243690490723
	mape = 5.07686185836792
	rmse = 17.60634994506836
